*ChatGPT의 검토에 따라 불필요한 부분 및 오류사항 수정:  
    - # 5. 최적 하이퍼파라미터 변수에 저장  
    - # 8. 선택된 변수만 사용하여 모델 학습 및 평가  
    - # 9. test data 예측  

## 1. EDA

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_absolute_error

In [2]:
# 1. 데이터 로드
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")  # 최종 검증용, 목표변수('Age') 없음.
submission_df = pd.read_csv("data/sample_submission.csv")

In [3]:
train_df.head()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,0,F,1.5250,1.2375,0.4125,32.630274,12.743100,6.577084,10.446791,10.0
1,1,M,1.8625,1.4250,0.5125,62.397250,28.335325,10.872033,18.852418,16.0
2,2,F,1.5500,1.2500,0.3875,31.808139,11.495722,7.271647,9.355335,14.0
3,3,M,1.4375,1.1125,0.3375,28.066005,13.877080,6.194366,7.796112,9.0
4,4,I,1.2875,1.0250,0.3000,15.748147,6.137667,3.472814,4.677668,8.0


In [4]:
train_df.describe()

,id,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,7499.500000,1.316742,1.023813,0.347326,23.372701,10.104499,5.040622,6.704216,9.966800
std,4330.271354,0.287869,0.237697,0.091335,12.754705,5.691158,2.805236,3.598253,3.238065
min,0.000000,0.187500,0.150000,0.000000,0.056699,0.028349,0.014175,0.042524,1.000000
25%,3749.750000,1.150000,0.887500,0.287500,13.377420,5.698250,2.820775,3.827183,8.000000
50%,7499.500000,1.375000,1.075000,0.362500,23.657658,9.879801,4.904464,6.803880,10.000000
75%,11249.250000,1.537500,1.200000,0.412500,32.205032,14.033003,7.002326,9.071840,11.000000
max,14999.000000,1.950000,1.575000,0.700000,80.101512,48.477645,19.220961,24.564842,29.000000


- Shucked Weight: 껍질을 제외한 무게
- Viscera Weight: 내장 무게
- Shell Weight: 껍질 무게
- `Age`가 목표변수

In [5]:
# test.csv에서 원본 id 저장
test_ids = test_df["id"].copy()
test_df = test_df.drop(columns=["id"])  # 이후 분석을 위해 삭제

## 2. 데이터 전처리

- 주의: 데이터 전처리 시 train data와 test data에서 동일한 특성이 유지되어야 한다.

In [6]:
# 'Sex' 라벨 인코딩
label_encoder = LabelEncoder()
train_df["Sex"] = label_encoder.fit_transform(train_df["Sex"])
test_df["Sex"] = label_encoder.transform(test_df["Sex"])

- LableEncoder() : 범주형 데이터를 수치형으로 인코딩(변환).

- `fit()` : 데이터를 학습시키는 메서드
- `transform()` : fit을 기준으로 얻은 mean, variance에 맞춰 데이터 변형하는 메서드
    - 실제로 학습시킨 것을 적용하는 메서드라고 생각하면 된다.
- `fit_transform()` : `fit()`과 `transform()`을 합쳐놓은 것.

주의: 이건 train dataset에 대해서만 해야 하고, test dataset에는 fit/transform을 하면 안 된다!  
+) 데이터 전처리 단계(sklearn의 preprocessing 패키지!)에서 사용하는 함수들.

In [7]:
# Height가 0인 경우 평균값으로 대체
height_mean = train_df.loc[train_df["Height"] > 0, "Height"].mean()
train_df.loc[train_df["Height"] == 0, "Height"] = height_mean
test_df.loc[test_df["Height"] == 0, "Height"] = height_mean

In [8]:
# 새로운 특성 추가
train_df["Volume"] = train_df["Length"] * train_df["Diameter"] * train_df["Height"]
test_df["Volume"] = test_df["Length"] * test_df["Diameter"] * test_df["Height"]

train_df["Shucked Weight Ratio"] = train_df["Shucked Weight"] / train_df["Weight"]
test_df["Shucked Weight Ratio"] = test_df["Shucked Weight"] / test_df["Weight"]

train_df["Viscera Weight Ratio"] = train_df["Viscera Weight"] / train_df["Weight"]
test_df["Viscera Weight Ratio"] = test_df["Viscera Weight"] / test_df["Weight"]

train_df["Density"] = train_df["Weight"] / train_df["Volume"]
test_df["Density"] = test_df["Weight"] / test_df["Volume"]

train_df["Edible Weight Ratio"] = (train_df["Shucked Weight"] + train_df["Viscera Weight"]) / train_df["Weight"]
test_df["Edible Weight Ratio"] = (test_df["Shucked Weight"] + test_df["Viscera Weight"]) / test_df["Weight"]

In [9]:
# IQR 기반 이상치 제거 함수 정의 및 적용
def remove_outliers_iqr(df, cols):
    Q1 = df[cols].quantile(0.25)
    Q3 = df[cols].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[~((df[cols] < lower_bound) | (df[cols] > upper_bound)).any(axis=1)]

num_cols = train_df.select_dtypes(include=["float64"]).columns
train_df = remove_outliers_iqr(train_df, num_cols)

In [10]:
# 중복 데이터 제거 및 'id' 컬럼 삭제
train_df = train_df.drop_duplicates().drop(columns=["id"])

### Train data -> train / valid 분리

In [11]:
# X, y 분리
X = train_df.drop(columns=["Age"])
y = train_df["Age"]

In [12]:
# Train/Validation Split (8:2 비율)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)

### 데이터 정규화

In [13]:
# 3. 데이터 정규화 (MinMaxScaler)
scaler = MinMaxScaler()

In [14]:
# 훈련 데이터와 검증 데이터를 스케일링
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

### 최적의 하이퍼파라미터 찾기

In [15]:
# 4. GridSearch를 이용한 최적 하이퍼파라미터 찾기 (교차 검증 cv=5)
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 5, 10],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2, 4],
}

# 모델 정의
rf = RandomForestRegressor(random_state=42, n_jobs=-1)  
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1, verbose=1)
grid_search.fit(X_train_scaled, y_train)

best_params = grid_search.best_params_
print("Best Parameters:", best_params)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}


- `GridSearchCV()` : 머신러닝 모델의 하이퍼파라미터 튜닝을 자동화하는 방법 중 하나.
    - 하이퍼파라미터 튜닝
    - 그리드 탐색
    - 교차검증(Cross Validation) 적용
    - 최적의 하이퍼파라미터 선택

- 'Best Parameters:{}' 결과로 도출된 하이퍼파라미터의 값들이 최적의 조합.

In [16]:
# 5. 최적 하이퍼파라미터 변수에 저장
best_rf = RandomForestRegressor(**best_params, random_state=42, n_jobs=-1)  # 수정-밑의 모델 학습 코드 삭제

#### 이 밑으로는 ChatGPT의 도움을 받아서 작성됨

In [17]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error

from sklearn.feature_selection import SequentialFeatureSelector as SFS

# 6. Cross Validation 설정 (KFold 사용)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

In [18]:
# 7. MAE를 가장 작게 하는 변수 선택(전진선택법)
# 
sfs = SFS(
    rf,
    n_features_to_select="auto",  # 자동으로 최적의 변수 개수 선택
    direction="forward",  # 전진 선택법 ("backward"로 하면 후진 제거법)
    scoring="neg_mean_absolute_error",
    cv=5,
    n_jobs=-1
)

# 훈련 데이터로 속성(변수) 선택 실행
sfs = sfs.fit(X_train_scaled, y_train)

# 선택된 변수 출력
selected_idx = sfs.get_support(indices=True)
print("선택된 변수 인덱스:", selected_idx)
print("변수명: ", X.columns[[0, 7, 8, 9, 11, 12]].tolist())  # 선택된 컬럼명

선택된 변수 인덱스: [ 0  7  8  9 11 12]
변수명:  ['Sex', 'Shell Weight', 'Volume', 'Shucked Weight Ratio', 'Density', 'Edible Weight Ratio']


- forward=True: 전진선택법, forward=False로 변경하면 후진제거법 
- floating=False는 순수한 단계별 선택을 의미

- `.get_support()`: SFS에서 특성 선택 결과를 Boolean 마스크 또는 선택된 특성의 인덱스로 반환하는 역할.

In [19]:
# 8. 선택된 변수만 사용하여 모델 학습 및 평가
selected_idx_list = list(sfs.get_support(indices=True))  ## 수정(indices=True)

X_train_selected = X_train_scaled[:, selected_idx_list]
X_valid_selected = X_valid_scaled[:, selected_idx_list]

# RandomForest 모델 학습
best_rf.fit(X_train_selected, y_train)

# valid data에 대한 예측
y_pred = best_rf.predict(X_valid_selected)

# mae 계산
valid_mae = mean_absolute_error(y_valid, y_pred)
print("Validation Data MAE: {:.4f}".format(valid_mae))

Validation Data MAE: 1.1286


In [ ]:
# 9. test data 예측
# 해결 못 함ㅜ
# test_df = test_df[test_df.columns[[0, 7, 8, 9, 11, 12]]]  # 적합하지 않음. 실제로 test data는 특성이 다르기 때문에 무슨 컬럼이 인덱싱 될 지 모르니까!
selected_columns = test_df[selected_idx_list]
test_df_scaled = scaler.transform(test_df[selected_columns])  # 선택된 특성만 스케일링.
test_preds = best_rf.predict(test_df_scaled)

KeyError: "None of [Index([0, 7, 8, 9, 11, 12], dtype='int64')] are in the [columns]"

- 주의: test data에서는 `fit`이 아니라 `transform`만 해 주어야 한다.

In [89]:
# 제출 파일 생성 (원래 id 유지)
submission = pd.DataFrame({"id": test_ids, "Age": test_preds})
submission.to_csv("download/sample_submission.csv", index=False)
print("sample_submission.csv 파일 생성 완료!")

sample_submission.csv 파일 생성 완료!
